# Deploy & Run Online Inference

Now that we've trained the model, and we are (presumably) happy with the results of training, we can deploy the model to a Vertex AI endpoint and use online predictions in order to test out a sample datapoint.

In [ ]:
PROJECT_NAME = 'mwpmltr'
LOCATION = "us-central1"
MODEL_NAME = "beatles_automl_file_out_2200_tags"
TARGET_COLUMN = "Like_The_Beatles"

In [ ]:
from google.cloud import aiplatform

In [ ]:
def create_endpoint(
    project: str,
    display_name: str,
    location: str,
):
    """Create an Vertex AI Model Endpoint in the given project and location"""
    
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint

In [ ]:
# Note that you don't have to create an endpoint every time you run this notebook
# create_endpoint(PROJECT_NAME, f'{MODEL_NAME}_endpoint', LOCATION)

In [ ]:
def deploy_model(
    project: str,
    location: str,
    model_name: str,
    endpoint_name: str
):
    """
    model_name: A fully-qualified model resource name or model ID.
    endpoint_name: A fully-qualified endpoint resource name or endpoint ID.
    """

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_name=model_name)
    endpoint = aiplatform.Endpoint(endpoint_name=endpoint_name)

    model.deploy(
        endpoint=endpoint,
        machine_type="e2-standard-4"
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model

In [ ]:
deploy_model(
    PROJECT_NAME,
    LOCATION,
    "projects/55590906972/locations/us-central1/models/8112673977777782784",
    "projects/55590906972/locations/us-central1/endpoints/3602110043756953600"
)

Now that the model is deployed to the prediction endpoint, we will use our test data point and make an API call to the Vertex AI online inference service, in order to predict whether this user would like the Beatles or not.

In [ ]:
from typing import List, Dict

def predict_tabular_classification(
    project: str,
    location: str,
    endpoint_name: str,
    instances: List[Dict],
):
    """
    Args
        project: Your project ID or project number.
        location: Region where Endpoint is located. For example, 'us-central1'.
        endpoint_name: A fully qualified endpoint name or endpoint ID. Example: "projects/123/locations/us-central1/endpoints/456" or
               "456" when project and location are initialized or passed.
        instances: A list of one or more instances (examples) to return a prediction for.
    """
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint_name)

    response = endpoint.predict(instances=instances)

    for prediction_ in response.predictions:
        print(prediction_)

In [ ]:
import pandas as pd

inference_sample = pd.read_feather('test_data/inference_sample.feather')

In [ ]:
for index, row in inference_sample.iterrows():
    instance = json.loads(row.astype(str).to_json())
    inference_results = predict_tabular_classification(PROJECT_NAME, LOCATION, 'projects/55590906972/locations/us-central1/endpoints/3602110043756953600', [instance])

In [ ]:
import json

with open('sample_request.json', 'w') as outfile:
    outfile.write(inference_sample.iloc[0].astype(str).to_json())


### Undeploy Model from Vertex AI Endpoint

### Pricing Notes
Resources that incur costs
Answer: you pay for three main activities
- Training the model
    - Price per node hour of tabular classification is $21.252, so that's the charge I incur every time I train the AutoML Beatles Model
- Deploying the model to an endpoint (models must be deployed before they can make either online predictions or online evaluations)
    - You pay for each model deployed to an endpoint, even if no prediction is made
    - Must undeploy your model to stop incurring further charges
    - Models that are not deployed or have failed to deploy are not charged
- Using the model to make predictions; this is for both batch and online predictions (which I think is BS, since we're also paying to host the model at an endpoint, but whatever)